# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 2:00PM,239016,18,INNOVA-8112583,Innova Softgel LLC,Released
1,Jun 7 2022 1:48PM,239014,10,MET210958,"Methapharm, Inc.",Released
2,Jun 7 2022 1:48PM,239014,10,MET210960,"Methapharm, Inc.",Released
3,Jun 7 2022 1:48PM,239014,10,MET210963,"Methapharm, Inc.",Released
4,Jun 7 2022 1:48PM,239014,10,MET210968,"Methapharm, Inc.",Released
5,Jun 7 2022 1:48PM,239014,10,MET210977,"Methapharm, Inc.",Released
6,Jun 7 2022 1:48PM,239014,10,MET210985,"Methapharm, Inc.",Released
7,Jun 7 2022 1:48PM,239014,10,MET210986,"Methapharm, Inc.",Released
8,Jun 7 2022 1:48PM,239014,10,MET210987,"Methapharm, Inc.",Released
9,Jun 7 2022 1:28PM,239011,12,U0931,"Uniderm USA, Inc",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,239009,Released,13
35,239010,Released,1
36,239011,Executing,1
37,239014,Released,8
38,239016,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239009,NaN,NaN,13.0
239010,NaN,NaN,1.0
239011,NaN,1.0,NaN
239014,NaN,NaN,8.0
239016,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238907,3.0,10.0,0.0
238909,0.0,0.0,3.0
238911,0.0,0.0,1.0
238914,0.0,1.0,0.0
238916,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238907,3,10,0
238909,0,0,3
238911,0,0,1
238914,0,1,0
238916,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238907,3,10,0
1,238909,0,0,3
2,238911,0,0,1
3,238914,0,1,0
4,238916,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238907,3,10,
1,238909,,,3
2,238911,,,1
3,238914,,1,
4,238916,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc."
9,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc"
10,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc."
23,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC"
24,Jun 7 2022 1:10PM,239007,20,Alumier Labs Inc.
40,Jun 7 2022 12:57PM,239004,10,Eywa Pharma Inc.
41,Jun 7 2022 12:49PM,239003,21,"NBTY Global, Inc."
42,Jun 7 2022 12:49PM,239002,21,"NBTY Global, Inc."
43,Jun 7 2022 12:39PM,238997,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,,,1
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",,,8
2,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",,1,
3,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",,,13
4,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",,,1
5,Jun 7 2022 1:10PM,239007,20,Alumier Labs Inc.,,,16
6,Jun 7 2022 12:57PM,239004,10,Eywa Pharma Inc.,,,1
7,Jun 7 2022 12:49PM,239003,21,"NBTY Global, Inc.",,,1
8,Jun 7 2022 12:49PM,239002,21,"NBTY Global, Inc.",,,1
9,Jun 7 2022 12:39PM,238997,10,Bio-PRF,,13,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,1,,
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8,,
2,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",,1,
3,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13,,
4,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",1,,
5,Jun 7 2022 1:10PM,239007,20,Alumier Labs Inc.,16,,
6,Jun 7 2022 12:57PM,239004,10,Eywa Pharma Inc.,1,,
7,Jun 7 2022 12:49PM,239003,21,"NBTY Global, Inc.",1,,
8,Jun 7 2022 12:49PM,239002,21,"NBTY Global, Inc.",1,,
9,Jun 7 2022 12:39PM,238997,10,Bio-PRF,8,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,1,,
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8,,
2,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",,1,
3,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13,,
4,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,1.0,NaN,NaN
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8.0,NaN,NaN
2,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",NaN,1.0,NaN
3,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13.0,NaN,NaN
4,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 2:00PM,239016,18,Innova Softgel LLC,1.0,0.0,0.0
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8.0,0.0,0.0
2,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",0.0,1.0,0.0
3,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13.0,0.0,0.0
4,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1911826,24.0,19.0,0.0
12,477995,2.0,1.0,0.0
15,716824,45.0,35.0,0.0
16,477814,0.0,20.0,6.0
17,238911,1.0,0.0,0.0
18,478026,2.0,0.0,0.0
19,716929,2.0,1.0,0.0
20,1672780,122.0,17.0,0.0
21,716921,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1911826,24.0,19.0,0.0
1,12,477995,2.0,1.0,0.0
2,15,716824,45.0,35.0,0.0
3,16,477814,0.0,20.0,6.0
4,17,238911,1.0,0.0,0.0
5,18,478026,2.0,0.0,0.0
6,19,716929,2.0,1.0,0.0
7,20,1672780,122.0,17.0,0.0
8,21,716921,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,24.0,19.0,0.0
1,12,2.0,1.0,0.0
2,15,45.0,35.0,0.0
3,16,0.0,20.0,6.0
4,17,1.0,0.0,0.0
5,18,2.0,0.0,0.0
6,19,2.0,1.0,0.0
7,20,122.0,17.0,0.0
8,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,24.0
1,12,Released,2.0
2,15,Released,45.0
3,16,Released,0.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
Executing,19.0,1.0,35.0,20.0,0.0,0.0,1.0,17.0,0.0
Released,24.0,2.0,45.0,0.0,1.0,2.0,2.0,122.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,1.0,35.0,20.0,0.0,0.0,1.0,17.0,0.0
2,Released,24.0,2.0,45.0,0.0,1.0,2.0,2.0,122.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,19.0,1.0,35.0,20.0,0.0,0.0,1.0,17.0,0.0
2,Released,24.0,2.0,45.0,0.0,1.0,2.0,2.0,122.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()